In [48]:
# Google Drive Setup
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Old version: simple string matching based on whether Solution Name in one run file is in the second file

In [49]:

# Re-running the initial comparison to redefine the variables
import pandas as pd

# Load the two CSV files for comparison again
file_path_1 = '/content/drive/MyDrive/resilience-tech-project/input/climate_adaptation_solutions_1.csv'
file_path_2 = '/content/drive/MyDrive/resilience-tech-project/input/climate_adaptation_solutions_2.csv'

solutions_run_1 = pd.read_csv(file_path_1)
solutions_run_2 = pd.read_csv(file_path_2)

# Normalize technologies and descriptions for better comparison
solutions_run_1['Technologies'] = solutions_run_1['Technologies'].str.lower().str.strip()
solutions_run_2['Technologies'] = solutions_run_2['Technologies'].str.lower().str.strip()

solutions_run_1['Description'] = solutions_run_1['Description'].str.lower().str.strip()
solutions_run_2['Description'] = solutions_run_2['Description'].str.lower().str.strip()

# Identify the unique solutions again
unique_to_run_1 = solutions_run_1[~solutions_run_1['Solution Name'].isin(solutions_run_2['Solution Name'])]
unique_to_run_2 = solutions_run_2[~solutions_run_2['Solution Name'].isin(solutions_run_1['Solution Name'])]

# Compare the technologies and descriptions for unique solutions
unique_to_run_1_comparison = unique_to_run_1[['Solution Name', 'Technologies', 'Description']]
unique_to_run_2_comparison = unique_to_run_2[['Solution Name', 'Technologies', 'Description']]

# Save these unique entries to files for further review
unique_to_run_1_comparison_file = '/content/drive/MyDrive/resilience-tech-project/output/unique_to_run_1_comparison.csv'
unique_to_run_2_comparison_file = '/content/drive/MyDrive/resilience-tech-project/output/unique_to_run_2_comparison.csv'

unique_to_run_1_comparison.to_csv(unique_to_run_1_comparison_file, index=False)
unique_to_run_2_comparison.to_csv(unique_to_run_2_comparison_file, index=False)

(unique_to_run_1_comparison_file, unique_to_run_2_comparison_file)



('/content/drive/MyDrive/resilience-tech-project/output/unique_to_run_1_comparison.csv',
 '/content/drive/MyDrive/resilience-tech-project/output/unique_to_run_2_comparison.csv')

In [50]:
unique_to_run_1


,Source,Category,Climate Problem,Solution Name,Technologies,Description
0,uploaded_pdf.pdf,Geological Hazards,Snow and ice,Snow and Ice Management Technologies,"autonomous vehicles, gps technology, iot sensors",implementing autonomous snow plows equipped wi...
1,uploaded_pdf.pdf,Water-Related Issues,Coastal flooding (incl. sea level rise),Resilient Coastal Infrastructure,"structural engineering technologies, resilienc...",this solution focuses on designing and constru...
2,uploaded_pdf.pdf,Geological Hazards,Fire weather (risk of wildfires),Wildfire Risk Assessment Tools,"remote sensing, machine learning, gis, data an...",utilizing satellite imagery and data analytics...
3,uploaded_pdf.pdf,NaN,Multiple climate-related risks,Community Engagement Platforms,"online forums, mobile applications, social med...",these digital platforms facilitate communicati...
4,uploaded_pdf.pdf,Geological Hazards,Fire weather (risk of wildfires),Wildfire Risk Assessment Tools,"big data analytics, machine learning, remote s...",utilizing advanced modeling and predictive ana...
...,...,...,...,...,...,...
127,uploaded_pdf.pdf,"Multiple categories (Extreme Weather Events, W...",Awareness and collaboration for enhanced resil...,Resilience Strategy Launch Event,"event management software, live streaming tool...",the launch event is designed to bring together...
128,uploaded_pdf.pdf,Extreme Weather Events,Compromised emergency response during extreme ...,Safe Havens for First Responders,"communication systems, energy backup systems, ...",by establishing reliable community resiliency ...
129,uploaded_pdf.pdf,Community and Neighborhoods,Various resilience challenges (indirectly rela...,Climate Resilience Workshops,"virtual meeting platforms, educational softwar...",hosting workshops to educate the community on ...
130,uploaded_pdf.pdf,Water-Related Issues,"Urban flooding, Extreme weather events",Climate Resilience in Asset Management Framework,"geographic information systems (gis), climate ...",integrating climate resilience into asset mana...


In [51]:
unique_to_run_2.head()


,Source,Category,Climate Problem,Solution Name,Technologies,Description
0,uploaded_pdf.pdf,Water-Related Issues; Extreme Weather Events,Increased water demand and urban heat,Sustainable Urban Mobility Solutions,"electric vehicles, mobile apps for transportat...",implementing sustainable urban mobility soluti...
1,uploaded_pdf.pdf,Geological Hazards,Fire weather (risk of wildfires),Fire Weather Monitoring System,"satellite imaging, meteorological sensors, ale...",a monitoring system that utilizes satellite da...
2,uploaded_pdf.pdf,Extreme Weather Events,"Hurricanes, cyclones, and extreme wind",Early Warning Systems,"satellite imagery, machine learning, communica...",technology that monitors atmospheric condition...
3,uploaded_pdf.pdf,Geological Hazards,Mass movement due to extreme precipitation,Landslide Monitoring System,"ground-penetrating radar, satellite imagery, d...",this system employs ground-penetrating radar a...
4,uploaded_pdf.pdf,Water-Related Issues,Coastal flooding and river flooding,Flood Risk Management Applications,"geographic information systems (gis), hydrauli...",flood risk management applications utilize gis...


New version: using TF-IDF (Term Frequency-Inverse Document Frequency) to compare the runs.

**TF-IDF (Term Frequency-Inverse Document Frequency)** is a technique to measure the importance of words in a document relative to a collection of documents. It assigns higher weights to terms that appear frequently in a single document but rarely across others, helping capture distinctive words for comparison. TF-IDF vectors are then used to calculate cosine similarity, a metric that measures the angle between two text vectors, where closer angles (higher similarity scores) indicate more similar texts.

**Function Summary:** The function *identify_unique_solutions* takes two datasets, uses the `Solution Name` and optionally `Technologies` (if include_tech = True) columns, and computes cosine similarity based on their TF-IDF vectors. It uses a threshold to filter unique solutions, outputting only those entries from each dataset that don't exceed the similarity threshold when compared to the other dataset.




In [65]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load the two CSV files for comparison again
file_path_1 = '/content/drive/MyDrive/resilience-tech-project/input/climate_adaptation_solutions_1.csv'
file_path_2 = '/content/drive/MyDrive/resilience-tech-project/input/climate_adaptation_solutions_2.csv'

solutions_run_1 = pd.read_csv(file_path_1)
solutions_run_2 = pd.read_csv(file_path_2)

# Normalize solution names and technologies for better comparison
solutions_run_1['Solution Name'] = solutions_run_1['Solution Name'].str.lower().str.strip()
solutions_run_1['Technologies'] = solutions_run_1['Technologies'].str.lower().str.strip()
solutions_run_2['Solution Name'] = solutions_run_2['Solution Name'].str.lower().str.strip()
solutions_run_2['Technologies'] = solutions_run_2['Technologies'].str.lower().str.strip()

# Define a function to calculate similarity and filter unique entries
def identify_unique_solutions(df1, df2, similarity_threshold, include_tech):
    # Combine 'Solution Name' and 'Technologies' columns for similarity comparison if specified
    if include_tech:
      df1['combined'] = df1['Solution Name'] + ' ' + df1['Technologies']
      df2['combined'] = df2['Solution Name'] + ' ' + df2['Technologies']
    # Otherwise just focus on solution names
    else:
      df1['combined'] = df1['Solution Name']
      df2['combined'] = df2['Solution Name']

    # Apply TF-IDF vectorization to combined text columns
    vectorizer = TfidfVectorizer().fit(df1['combined'].tolist() + df2['combined'].tolist())
    tfidf_matrix_1 = vectorizer.transform(df1['combined'])
    tfidf_matrix_2 = vectorizer.transform(df2['combined'])

    # Compute cosine similarity between all rows
    similarity_matrix = cosine_similarity(tfidf_matrix_1, tfidf_matrix_2)

    # Identify unique solutions by threshold
    unique_indices_run_1 = [i for i in range(similarity_matrix.shape[0]) if max(similarity_matrix[i]) < similarity_threshold]
    unique_indices_run_2 = [j for j in range(similarity_matrix.shape[1]) if max(similarity_matrix[:, j]) < similarity_threshold]

    # Identify common solutions by threshold
    common_indices_run = [i for i in range(similarity_matrix.shape[0]) if max(similarity_matrix[i]) > similarity_threshold]
    # Filter unique solutions
    unique_to_run_1 = df1.iloc[unique_indices_run_1]
    unique_to_run_2 = df2.iloc[unique_indices_run_2]
    common_to_run = df1.iloc[common_indices_run]

    return unique_to_run_1[['Solution Name', 'Technologies']], unique_to_run_2[['Solution Name', 'Technologies']], common_to_run[['Solution Name', 'Technologies']]


# Identify unique solutions with a similarity threshold of 0.8 (adjust as needed)
threshold_tech=0.5
threshold_nontech = 0.5
tfidf_unique_to_run_1_comparison, tfidf_unique_to_run_2_comparison, tfidf_common_two_runs_comparison = identify_unique_solutions(solutions_run_1, solutions_run_2, threshold_nontech, False)
tfidf_unique_to_run_1_comparison_tech, tfidf_unique_to_run_2_comparison_tech, tfidf_common_two_runs_comparison_tech = identify_unique_solutions(solutions_run_1, solutions_run_2, threshold_tech, True)

# Save these unique entries to files for further review
tfidf_unique_to_run_1_comparison_file = '/content/drive/MyDrive/resilience-tech-project/output/tfidf_unique_to_run_1_comparison.csv'
tfidf_unique_to_run_2_comparison_file = '/content/drive/MyDrive/resilience-tech-project/output/tfidf_unique_to_run_2_comparison.csv'
tfidf_common_two_runs_comparison_file = '/content/drive/MyDrive/resilience-tech-project/output/tfidf_common_solutions.csv'

tfidf_unique_to_run_1_comparison.to_csv(tfidf_unique_to_run_1_comparison_file, index=False)
tfidf_unique_to_run_2_comparison.to_csv(tfidf_unique_to_run_2_comparison_file, index=False)
tfidf_common_two_runs_comparison.to_csv(tfidf_common_two_runs_comparison_file, index=False)

tfidf_unique_to_run_1_comparison_file_tech = '/content/drive/MyDrive/resilience-tech-project/output/tfidf_unique_to_run_1_tech_comparison.csv'
tfidf_unique_to_run_2_comparison_file_tech = '/content/drive/MyDrive/resilience-tech-project/output/tfidf_unique_to_run_2_tech_comparison.csv'
tfidf_common_two_runs_comparison_file_tech = '/content/drive/MyDrive/resilience-tech-project/output/tfidf_common_tech_solutions.csv'

tfidf_unique_to_run_1_comparison_tech.to_csv(tfidf_unique_to_run_1_comparison_file_tech, index=False)
tfidf_unique_to_run_2_comparison_tech.to_csv(tfidf_unique_to_run_2_comparison_file_tech, index=False)
tfidf_common_two_runs_comparison_tech.to_csv(tfidf_common_two_runs_comparison_file_tech, index=False)

(tfidf_unique_to_run_1_comparison_file, tfidf_unique_to_run_2_comparison_file, tfidf_common_two_runs_comparison_file)


('/content/drive/MyDrive/resilience-tech-project/output/tfidf_unique_to_run_1_comparison.csv',
 '/content/drive/MyDrive/resilience-tech-project/output/tfidf_unique_to_run_2_comparison.csv',
 '/content/drive/MyDrive/resilience-tech-project/output/tfidf_common_solutions.csv')

In [66]:
print("Solution matching based on Solution Name")
print("Total number of solutions for run 1: ", len(solutions_run_1))
print("Unique solutions from run 1, for Solution Name column, with a threshold = ", threshold_nontech, " is ", len(tfidf_unique_to_run_1_comparison))
print(round(len(tfidf_unique_to_run_1_comparison)/len(solutions_run_1),2) * 100, " percent of solutions for run 1 is unique")

print("Total number of solutions for run 2: ", len(solutions_run_2))
print("Unique solutions from run 2, for Solution Name column, with a threshold = ", threshold_nontech, " is ", len(tfidf_unique_to_run_2_comparison))

print(round(len(tfidf_common_two_runs_comparison)/len(solutions_run_1),2) * 100, " percent of solutions common for run 1 and run 2")


Solution matching based on Solution Name
Total number of solutions for run 1:  133
Unique solutions from run 1, for Solution Name column, with a threshold =  0.5  is  81
61.0  percent of solutions for run 1 is unique
Total number of solutions for run 2:  139
Unique solutions from run 2, for Solution Name column, with a threshold =  0.5  is  87
39.0  percent of solutions common for run 1 and run 2


In [67]:
print("Solution matching based on Solution Name and Technology description")
print("Total number of solutions for run 1: ", len(solutions_run_1))
print("Unique solutions from run 1, for combined Solution Name and Technology columns, with a threshold = ", threshold_tech, " is ", len(tfidf_unique_to_run_1_comparison_tech))
print(round(len(tfidf_unique_to_run_1_comparison)/len(solutions_run_1),2) * 100, " percent of solutions for run 1 is unique")

print("Total number of solutions for run 2: ", len(solutions_run_2))
print("Unique solutions from run 2, for combined Solution Name and Technology columns, with a threshold = ", threshold_tech, " is ", len(tfidf_unique_to_run_2_comparison_tech))

print(round(len(tfidf_common_two_runs_comparison_tech)/len(solutions_run_1),2) * 100, " percent of solutions common for run 1 and run 2")


Solution matching based on Solution Name and Technology description
Total number of solutions for run 1:  133
Unique solutions from run 1, for combined Solution Name and Technology columns, with a threshold =  0.5  is  89
61.0  percent of solutions for run 1 is unique
Total number of solutions for run 2:  139
Unique solutions from run 2, for combined Solution Name and Technology columns, with a threshold =  0.5  is  98
33.0  percent of solutions common for run 1 and run 2


In [55]:
tfidf_unique_to_run_1_comparison

,Solution Name,Technologies
5,community cooling centers,"solar panels, energy-efficient hvac systems, c..."
8,accessible resource mapping applications,"gis mapping, mobile applications, crowd-source..."
9,data analytics tools for vulnerability assessment,"big data analytics, machine learning algorithm..."
10,resilient housing initiatives through smart bu...,"building automation systems, energy management..."
15,coastal resilience modeling tools,"simulation modeling software, gis, remote sens..."
...,...,...
126,resilience building grantees program,"grant management software, data tracking syste..."
127,resilience strategy launch event,"event management software, live streaming tool..."
128,safe havens for first responders,"communication systems, energy backup systems, ..."
130,climate resilience in asset management framework,"geographic information systems (gis), climate ..."


In [56]:
tfidf_unique_to_run_2_comparison

,Solution Name,Technologies
0,sustainable urban mobility solutions,"electric vehicles, mobile apps for transportat..."
3,landslide monitoring system,"ground-penetrating radar, satellite imagery, d..."
4,flood risk management applications,"geographic information systems (gis), hydrauli..."
5,predictive health risk management system,"data analytics, machine learning, mobile healt..."
6,vertical resilience project,"building energy management systems, green buil..."
...,...,...
130,integrated community leadership,"collaboration platforms, social media engageme..."
131,resilience office,"project management tools, stakeholder engageme..."
132,emergency command centers,"communication technology, emergency management..."
133,hosting global resilience events,"event management software, streaming technolog..."


In [57]:
tfidf_common_two_runs_comparison

,Solution Name,Technologies
0,snow and ice management technologies,"autonomous vehicles, gps technology, iot sensors"
1,resilient coastal infrastructure,"structural engineering technologies, resilienc..."
2,wildfire risk assessment tools,"remote sensing, machine learning, gis, data an..."
3,community engagement platforms,"online forums, mobile applications, social med..."
4,wildfire risk assessment tools,"big data analytics, machine learning, remote s..."
6,wildfire risk mitigation technology,"remote sensing, satellite imagery, early warni..."
7,emergency communication networks,"mobile applications, cloud computing, satellit..."
11,coastal resilience monitoring systems,"satellite remote sensing, iot sensors, gis, da..."
12,climate-positive villiers island development,"sustainable design practices, green infrastruc..."
13,coastal resilience monitoring system,"iot sensors, predictive modeling, gis, real-ti..."
